# Notebook 1 — Seed the SQLite DB (exams + questions) and sanity checks

**Goal:** create schema, load CSVs, and verify data.

In [2]:
import os 
os.chdir(os.getcwd().split('\\notebooks')[0])
main = os.getcwd()

In [4]:
# 1) Setup
import sqlite3, pandas as pd, re
from pathlib import Path

DB_PATH = main + "\\data\\temporal\\exams.db"
#DB_PATH.parent.mkdir(parents=True, exist_ok=True)

def connect(db_path=DB_PATH):
    con = sqlite3.connect(db_path)
    con.row_factory = sqlite3.Row
    return con

In [5]:
# 2) Create schema (users, exams, questions, attempts)
def create_schema(con):
    cur = con.cursor()
    cur.executescript("""
    CREATE TABLE IF NOT EXISTS users(
      user_id   INTEGER PRIMARY KEY AUTOINCREMENT,
      username  TEXT NOT NULL UNIQUE
    );

    CREATE TABLE IF NOT EXISTS exams(
      exam_id    TEXT PRIMARY KEY,
      exam_type  TEXT,
      date       TEXT,
      year       INTEGER
    );

    CREATE TABLE IF NOT EXISTS questions(
      exercise_id TEXT PRIMARY KEY,
      exam_id     TEXT,
      question    TEXT,
      solution    TEXT,
      topic_pred  TEXT,
      FOREIGN KEY (exam_id) REFERENCES exams(exam_id)
    );

    CREATE TABLE IF NOT EXISTS attempts(
      attempt_id        INTEGER PRIMARY KEY AUTOINCREMENT,
      ts                REAL DEFAULT (strftime('%s','now')),
      user_id           INTEGER NOT NULL,
      exercise_id       TEXT    NOT NULL,
      score             REAL,
      correct           INTEGER,
      reasons           TEXT,
      hint              TEXT,
      FOREIGN KEY (user_id)     REFERENCES users(user_id),
      FOREIGN KEY (exercise_id) REFERENCES questions(exercise_id)
    );
    """ )
    con.commit()

with connect() as con:
    create_schema(con)
print("Schema OK →", DB_PATH)

Schema OK → d:\ESS\ocr_math_q\data\temporal\exams.db


In [ ]:
# 3) Seed from CSVs (put your CSVs in ./data): exams.csv, questions.csv
EXAMS_CSV = Path("data/exams.csv")
QUESTIONS_CSV = Path("data/questions.csv")

def seed_exams(con, csv_path=EXAMS_CSV):
    df = pd.read_csv(csv_path)
    df[["exam_id","exam_type","date","year"]].to_sql("exams", con, if_exists="append", index=False)

def seed_questions(con, csv_path=QUESTIONS_CSV):
    df = pd.read_csv(csv_path)
    df[["exercise_id","exam_id","question","solution","topic_pred"]].to_sql("questions", con, if_exists="append", index=False)

with connect() as con:
    if EXAMS_CSV.exists():     seed_exams(con)
    if QUESTIONS_CSV.exists(): seed_questions(con)

print("Seed complete.")

In [ ]:
# 4) If you don't have CSVs yet, insert a tiny sample so the next notebooks run
sample_exams = pd.DataFrame([
    {"exam_id":"General_2025-08-29","exam_type":"General","date":"2025-08-29","year":2025}
])
sample_questions = pd.DataFrame([
    {"exercise_id":"General_2025-08-29_Exercise_1","exam_id":"General_2025-08-29",
     "question":"State the Riesz representation theorem.",
     "solution":"Every bounded linear functional on a Hilbert space is inner product with a unique vector.",
     "topic_pred":"linear functional"},
    {"exercise_id":"General_2025-08-29_Exercise_2","exam_id":"General_2025-08-29",
     "question":"Show f is a contraction and apply Banach.",
     "solution":"Compute Lipschitz constant L<1 and then use Banach contraction principle.",
     "topic_pred":"Banach contraction"},
])

with connect() as con:
    if con.execute("SELECT COUNT(*) FROM exams").fetchone()[0] == 0:
        sample_exams.to_sql("exams", con, if_exists="append", index=False)
    if con.execute("SELECT COUNT(*) FROM questions").fetchone()[0] == 0:
        sample_questions.to_sql("questions", con, if_exists="append", index=False)

print("Sample rows ensured.")

In [ ]:
# 5) Sanity checks
with connect() as con:
    print("exams:", con.execute("SELECT COUNT(*) FROM exams").fetchone()[0])
    print("questions:", con.execute("SELECT COUNT(*) FROM questions").fetchone()[0])

    # fetch one question
    row = con.execute("""
      SELECT q.exercise_id, q.question, q.solution, q.topic_pred, e.date, e.exam_type
      FROM questions q LEFT JOIN exams e ON e.exam_id=q.exam_id
      LIMIT 1
    """ ).fetchone()
    dict(row)